In [1]:
!pip install -q transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from transformers import pipeline

In [4]:
# to import files from googledrive
from google.colab import drive


drive.mount('/content/gdrive/', force_remount=True)
sample_size_dir = 'gdrive/My Drive/Sample size for NLP'

Mounted at /content/gdrive/


In [5]:
df = pd.read_csv(sample_size_dir+'/data_jaya/41401_combined_data_1000.csv')
df.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,label
0,360799,31329,135605,4.0,76527,31187,2100-09-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2100-8-29**] Dischar...,0
1,219682,19673,155584,6.0,27650,3523,2195-06-01,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2195-5-27**] ...,0
2,348049,30315,154805,8.0,V290,32549,2183-11-10,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2183-10-19**] Discha...,0
3,537465,74768,144932,1.0,41071,38111,2134-07-15,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2134-7-11**] ...,0
4,560154,79735,156817,4.0,7291,41370,2134-03-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2134-3-10**] ...,0


# Approach one

Options:

"distilbert/distilbert-base-uncased-distilled-squad"

"deepset/roberta-base-squad2"

"google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"





In [4]:
# Load a pre-trained question answering pipeline
qa_pipeline = pipeline("question-answering", model="google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
# Question to compare with
question = "Has this patient been diagnosed with coronary atherosclerosis of the native coronary artery?"

In [6]:
# Function to get yes/no and probabilities
def get_answer(context, question):
    result = qa_pipeline(question=question, context=context)
    yes_prob = result['score'] if result['answer'].lower() in ['yes', 'y', '1'] else 1 - result['score']
    label = 1 if result['answer'].lower() in ['yes', 'y', '1'] else 0
    return label, yes_prob

In [7]:
# Calculate yes/no and probabilities for each row
df['answer_QA'], df['probability'] = zip(*df['TEXT'].apply(lambda x: get_answer(x, question)))

In [8]:
df.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,label,answer_QA,probability
0,360799,31329,135605,4.0,76527,31187,2100-09-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2100-8-29**] Dischar...,0,0,0.417378
1,219682,19673,155584,6.0,27650,3523,2195-06-01,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2195-5-27**] ...,0,0,0.996170
2,348049,30315,154805,8.0,V290,32549,2183-11-10,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2183-10-19**] Discha...,0,0,0.990692
3,537465,74768,144932,1.0,41071,38111,2134-07-15,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2134-7-11**] ...,0,0,0.981914
4,560154,79735,156817,4.0,7291,41370,2134-03-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2134-3-10**] ...,0,0,0.937530


In [9]:
df.to_csv(sample_size_dir+'/data_jaya/41401_combined_data_1000_w_QA_probs_bertlarge.csv')

# Approach two

another approach

from https://medium.com/saarthi-ai/build-a-smart-question-answering-system-with-fine-tuned-bert-b586e4cfa5f5

In [6]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer


In [22]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad', return_dict=False)

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
# example

question = '''Why was the student group called "the Methodists?"'''

paragraph = ''' The movement which would become The United Methodist Church began in the mid-18th century within the Church of England.
            A small group of students, including John Wesley, Charles Wesley and George Whitefield, met on the Oxford University campus.
            They focused on Bible study, methodical study of scripture and living a holy life.
            Other students mocked them, saying they were the "Holy Club" and "the Methodists", being methodical and exceptionally detailed in their Bible study, opinions and disciplined lifestyle.
            Eventually, the so-called Methodists started individual societies or classes for members of the Church of England who wanted to live a more religious life. '''


In [78]:
df.tail()

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,label,length,TEXT_trunc,answer,corrected_answer
995,78430,7045,150235,10.0,4019,53002,2159-02-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2159-1-6**] D...,1,7408,Admission Date: [**2159-1-6**] D...,corona ##ry at ##her ##os ##cle ##rosis of the...,coronary atherosclerosis of the native coronar...
996,151984,13559,178090,8.0,4019,49141,2177-11-26,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2177-11-25**] Discha...,1,24712,Admission Date: [**2177-11-25**] Discha...,history of present illness : this is a 70 year...,history of present illness : this is a 70 year...
997,201153,18068,135796,13.0,4019,120,2116-01-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2115-12-23**] ...,1,11606,Admission Date: [**2115-12-23**] ...,"chief complaint : cad , 3 vessel cad , ao ##rt...","chief complaint : cad , 3 vessel cad , aortic ..."
998,509188,68307,170209,9.0,4019,37775,2162-12-03,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-11-10**] ...,1,19418,Admission Date: [**2162-11-10**] ...,patient recorded as having no known all ##er #...,patient recorded as having no known allergies ...
999,452906,55992,161385,7.0,4019,2810,2153-08-24,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2153-8-19**] ...,1,8322,Admission Date: [**2153-8-19**] ...,,


In [44]:
df['length'] = df['TEXT'].apply(len)

In [46]:
df_512 = df[df['length'] < 512].copy()

In [47]:
df_512.head() # there are no texts less than 512

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,label,length


In [62]:
# truncate text from beginning

df['TEXT_trunc'] = df['TEXT'].apply(lambda x: x[:512])

In [64]:
df['TEXT_trunc'][995]

'Admission Date:  [**2159-1-6**]              Discharge Date:   [**2159-2-9**]\n\nDate of Birth:  [**2087-4-17**]             Sex:   M\n\nService: MEDICINE\n\nAllergies:\nPatient recorded as having No Known Allergies to Drugs\n\nAttending:[**First Name3 (LF) 338**]\nChief Complaint:\nFailure to thrive\n\nMajor Surgical or Invasive Procedure:\nThoracentesis, pleuroscopy, chest tube placement\n\nHistory of Present Illness:\nHistory of Present Illness: Mr. [**Known lastname 109257**] is a 71yo male with PMH\nsignificant for CAD,'

In [66]:
# mimic example

question = '''Has this patient been diagnosed with coronary atherosclerosis of the native coronary artery?'''

paragraph = df['TEXT_trunc'][980]

In [67]:
encoding = tokenizer.encode_plus(text=question,text_pair=paragraph, add_special_tokens=True) # add_special is not correct - add_special_token

In [68]:
inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

In [69]:
start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

In [70]:
type(start_scores)

torch.Tensor

In [71]:
start_index = torch.argmax(start_scores)

end_index = torch.argmax(end_scores)

answer = ' '.join(tokens[start_index:end_index+1])

In [72]:
answer

'history of present illness : 87 year old female with ao ##rti ##c ste ##nosis'

In [73]:
corrected_answer = ''

for word in answer.split():

    #If it's a subword token
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word

In [74]:
corrected_answer

' history of present illness : 87 year old female with aortic stenosis'

Run on entire df

In [75]:
def get_answer(paragraph):
    encoding = tokenizer.encode_plus(text=question, text_pair=paragraph, add_special_tokens=True)
    inputs = encoding['input_ids']
    sentence_embedding = encoding['token_type_ids']
    tokens = tokenizer.convert_ids_to_tokens(inputs)

    with torch.no_grad():
        start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    answer = ' '.join(tokens[start_index:end_index+1])

    corrected_answer = ''
    for word in answer.split():
        if word[0:2] == '##':
            corrected_answer += word[2:]
        else:
            corrected_answer += ' ' + word

    return answer, corrected_answer.strip()


In [76]:
df[['answer', 'corrected_answer']] = df['TEXT_trunc'].apply(lambda x: pd.Series(get_answer(x)))

In [77]:
df.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,label,length,TEXT_trunc,answer,corrected_answer
0,360799,31329,135605,4.0,76527,31187,2100-09-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2100-8-29**] Dischar...,0,4627,Admission Date: [**2100-8-29**] Dischar...,corona ##ry at ##her ##os ##cle ##rosis,coronary atherosclerosis
1,219682,19673,155584,6.0,27650,3523,2195-06-01,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2195-5-27**] ...,0,6924,Admission Date: [**2195-5-27**] ...,corona ##ry at ##her ##os ##cle ##rosis of the...,coronary atherosclerosis of the native coronar...
2,348049,30315,154805,8.0,V290,32549,2183-11-10,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2183-10-19**] Discha...,0,5969,Admission Date: [**2183-10-19**] Discha...,corona ##ry at ##her ##os ##cle ##rosis,coronary atherosclerosis
3,537465,74768,144932,1.0,41071,38111,2134-07-15,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2134-7-11**] ...,0,13038,Admission Date: [**2134-7-11**] ...,corona ##ry at ##her ##os ##cle ##rosis of the...,coronary atherosclerosis of the native coronar...
4,560154,79735,156817,4.0,7291,41370,2134-03-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2134-3-10**] ...,0,11428,Admission Date: [**2134-3-10**] ...,patient had not seen any doctor for her 2 weeks,patient had not seen any doctor for her 2 weeks


In [79]:
df.to_csv(sample_size_dir+'/data_jaya/41401_combined_data_1000_w_QA_probs_bertlarge_answers.csv')